In [1]:
pip install datasets


  Using cached datasets-2.11.0-py3-none-any.whl (468 kB)
  Using cached responses-0.18.0-py3-none-any.whl (38 kB)
  Using cached fsspec-2023.4.0-py3-none-any.whl (153 kB)
  Using cached aiohttp-3.8.4-cp38-cp38-win_amd64.whl (324 kB)
  Using cached huggingface_hub-0.13.4-py3-none-any.whl (200 kB)
  Using cached multiprocess-0.70.14-py38-none-any.whl (132 kB)
  Using cached PyYAML-6.0-cp38-cp38-win_amd64.whl (155 kB)
  Using cached pyarrow-11.0.0-cp38-cp38-win_amd64.whl (20.6 MB)
  Using cached xxhash-3.2.0-cp38-cp38-win_amd64.whl (30 kB)
  Using cached dill-0.3.6-py3-none-any.whl (110 kB)
  Using cached aiosignal-1.3.1-py3-none-any.whl (7.6 kB)
  Using cached attrs-22.2.0-py3-none-any.whl (60 kB)
  Using cached yarl-1.8.2-cp38-cp38-win_amd64.whl (56 kB)
  Using cached multidict-6.0.4-cp38-cp38-win_amd64.whl (28 kB)
  Using cached frozenlist-1.3.3-cp38-cp38-win_amd64.whl (34 kB)
  Using cached async_timeout-4.0.2-py3-none-any.whl (5.8 kB)


In [2]:
from datasets import load_dataset
import pandas as pd
from pathlib import Path

c:\Users\NHI360\Desktop\ml-youtube-search\env\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
from datasets import load_dataset

# first download the dataset
data = load_dataset(
    'jamescalam/youtube-transcriptions',
    split='train'
)

new_data = []  # this will store adjusted data

window = 6  # number of sentences to combine
stride = 3  # number of sentences to 'stride' over, used to create overlap

for i in range(0, len(data), stride):
    i_end = min(len(data)-1, i+window)
    if data[i]['title'] != data[i_end]['title']:
        # in this case we skip this entry as we have start/end of two videos
        continue
    # create larger text chunk
    text = ' '.join(data[i:i_end]['text'])
    # add to adjusted data list
    new_data.append({
        'start': data[i]['start'],
        'end': data[i_end]['end'],
        'title': data[i]['title'],
        'text': text,
        'id': data[i]['id'],
        'url': data[i]['url'],
        'published': data[i]['published']
    })


Found cached dataset json (C:/Users/NHI360/.cache/huggingface/datasets/jamescalam___json/jamescalam--youtube-transcriptions-08d889f6a5386b9b/0.0.0/fe5dd6ea2639a6df622901539cb550cf8797e5a6b2dd7af1cf934bed8e233e6e)


In [4]:
j_df = pd.DataFrame(new_data)


In [5]:

j_df = j_df.drop(['published'],axis=1)
j_df.columns

Index(['start', 'end', 'title', 'text', 'id', 'url'], dtype='object')

In [6]:
j_df['start'] = j_df['start'].astype("float32")
j_df['end'] = j_df['end'].astype("float32")
j_df['duration'] = j_df['end']-j_df['start']
j_df = j_df.reindex(
            columns=["title", "url", "id", "start", "end", "duration", "text"]
    )

In [7]:
path = Path.cwd().parent
data_path = str(path / 'data_files' / 'videos_info' /'playlist_info' / 'jamescalam_youtube-transcriptions.csv')
j_df.to_csv(data_path,sep=',')

In [8]:
data =load_dataset('ashraq/youtube-transcription')

Found cached dataset youtube-transcription (C:/Users/NHI360/.cache/huggingface/datasets/ashraq___youtube-transcription/default/1.2.0/bec10e3d58fe13b3eee1ebefb584ecba03abfcced454bf26ad344ff60f261cfa)
100%|██████████| 1/1 [00:00<00:00, 53.20it/s]


In [9]:
a_df = pd.DataFrame(data['train'])

In [10]:
a_df.columns

Index(['video_id', 'title', 'text', 'start_timestamp', 'end_timestamp',
       'start_second', 'end_second', 'url', 'thumbnail'],
      dtype='object')

In [11]:
a_df = a_df.drop(['start_timestamp','end_timestamp','thumbnail'],axis=1)
a_df = a_df.rename(columns={'video_id':'id','start_second':'start','end_second':'end'})
a_df.columns

Index(['id', 'title', 'text', 'start', 'end', 'url'], dtype='object')

In [12]:
a_df['start'] = a_df['start'].astype("float32")
a_df['end'] = a_df['end'].astype("float32")
a_df['duration'] = a_df['end']-a_df['start']
a_df = a_df.reindex(
            columns=["title", "url", "id", "start", "end", "duration", "text"]
    )

In [13]:
path = Path.cwd().parent
data_path = str(path / 'data_files' / 'videos_info' /'playlist_info' / 'ashraq_youtube-transcription.csv')
a_df.to_csv(data_path,sep=',')

In [14]:
p_data = load_dataset('pinecone/yt-transcriptions')

Found cached dataset json (C:/Users/NHI360/.cache/huggingface/datasets/pinecone___json/pinecone--yt-transcriptions-4383a838c5c147e8/0.0.0/fe5dd6ea2639a6df622901539cb550cf8797e5a6b2dd7af1cf934bed8e233e6e)
100%|██████████| 1/1 [00:00<00:00, 35.17it/s]


In [15]:
p_df = pd.DataFrame(p_data['train'])

In [16]:
p_df.columns

Index(['video_id', 'text', 'start_second', 'end_second', 'url', 'title',
       'thumbnail'],
      dtype='object')

In [17]:
p_df = p_df.drop(['thumbnail'],axis=1)
p_df = p_df.rename(columns={'video_id':'id','start_second':'start','end_second':'end'})

In [18]:
p_df['start'] = p_df['start'].astype("float32")
p_df['end'] = p_df['end'].astype("float32")
p_df['duration'] = p_df['end']-p_df['start']
p_df = p_df.reindex(
            columns=["title", "url", "id", "start", "end", "duration", "text"]
    )

In [19]:
path = Path.cwd().parent
data_path = str(path / 'data_files' / 'videos_info' /'playlist_info' / 'pinecone_youtube-transcription.csv')
p_df.to_csv(data_path,sep=',')

In [31]:
#combinin all datasets
import pandas as pd
ml = pd.read_csv(r'C:\Users\NHI360\Desktop\ai-youtube-search\data_files\videos_info\playlist_info\Machine Learning.csv')
jamescalam = pd.read_csv(r"C:\Users\NHI360\Desktop\ai-youtube-search\data_files\videos_info\playlist_info\jamescalam_youtube-transcriptions.csv")
ashraq = pd.read_csv(r"C:\Users\NHI360\Desktop\ai-youtube-search\data_files\videos_info\playlist_info\ashraq_youtube-transcription.csv")
pinecone = pd.read_csv(r"C:\Users\NHI360\Desktop\ai-youtube-search\data_files\videos_info\playlist_info\pinecone_youtube-transcription.csv")

In [32]:
final_dataset = pd.concat([ml,ashraq,pinecone,jamescalam],axis=0,ignore_index=True)

In [33]:
final_dataset.columns
final_dataset = final_dataset.drop('Unnamed: 0',axis=1)



In [34]:
from pathlib import Path
path = Path.cwd().parent
data_path = str(path / 'data_files' / 'videos_info' /'playlist_info' / 'Dataset.csv')

final_dataset.to_csv(data_path, sep=',', index=False)

In [35]:
final_dataset

,title,url,id,start,end,duration,text
0,A Gentle Introduction to Machine Learning,https://www.youtube.com/watch?v=Gv9_4yMHFhI,Gv9_4yMHFhI,0.00,20.00,20.000000,Gonna start this stack quest with the silly s...
1,A Gentle Introduction to Machine Learning,https://www.youtube.com/watch?v=Gv9_4yMHFhI,Gv9_4yMHFhI,21.00,41.00,20.000000,"Note, this stack quest was originally prepare..."
2,A Gentle Introduction to Machine Learning,https://www.youtube.com/watch?v=Gv9_4yMHFhI,Gv9_4yMHFhI,42.00,69.00,27.000000,"If you like silly songs, are you interested i..."
3,A Gentle Introduction to Machine Learning,https://www.youtube.com/watch?v=Gv9_4yMHFhI,Gv9_4yMHFhI,69.00,97.00,28.000000,"Why not why? If you don't like silly songs, a..."
4,A Gentle Introduction to Machine Learning,https://www.youtube.com/watch?v=Gv9_4yMHFhI,Gv9_4yMHFhI,98.00,127.00,29.000000,"Otherwise, you might not like Stack Quest. Wh..."
...,...,...,...,...,...,...,...
90628,GLOM: How to represent part-whole hierarchies ...,https://youtu.be/cllFzkvrYmE,cllFzkvrYmE-t3716.04,3716.04,3769.80,53.760010,"already taken Sorry. I also by the way, also c..."
90629,GLOM: How to represent part-whole hierarchies ...,https://youtu.be/cllFzkvrYmE,cllFzkvrYmE-t3742.44,3742.44,3788.92,46.479980,It has a bunch of parts that are maybe not sup...
90630,GLOM: How to represent part-whole hierarchies ...,https://youtu.be/cllFzkvrYmE,cllFzkvrYmE-t3759.36,3759.36,3819.72,60.359863,"might be a bit tricky. And, but the rest and t..."
90631,GLOM: How to represent part-whole hierarchies ...,https://youtu.be/cllFzkvrYmE,cllFzkvrYmE-t3776.36,3776.36,3819.72,43.359863,"happens, maybe there's a CNN, maybe the CNN fe..."
